### **1. Сбор данных**

В качестве источника данных был выбран сайт https://spasibovsem.ru, в котором собраны отзывы на фильмы разных времен. Парсинг осуществлялся с помощью HHTP-протоколов модуля requests, а извлечение данных стало доступно благодаря модулю BeautifulSoup. Для предотвращения блокировки как реакции сайта на краулинг были приняты следующие меры:
1) Рандомный фальшивый браузер (UserAgent)
2) Промежутки time.sleep(<секунды>) между запросами

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import time

ua = UserAgent(verify_ssl=True)
session = requests.session()
header={'User-Agent': ua.random, 'referer':'https://www.google.com/'}

In [ ]:
all_info = pd.DataFrame(columns=['Name', 'Review', 'Rate'])
number_of_pages = 181
for i in range(1, number_of_pages+1):
    time.sleep(2)
    req = session.get(f'https://spasibovsem.ru/filmy-otzyvy/?page={i}', headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    items = soup.find_all('div', {'class': 'name'})
    for item in items:
        time.sleep(2)
        item_name = item.find('a').get_text()
        link = 'https://spasibovsem.ru' + item.find('a')['href']
        req = session.get(link, headers=header)
        item_page = BeautifulSoup(req.text, 'html.parser')
        review_links = item_page.find_all('div', {'class': 'full-item goto'})
        if isinstance(review_links, list) == False:
            review_links = [review_links]
        for review_link in review_links:
            time.sleep(3)
            review_link = review_link.find('a')['href']
            req_review = session.get(f'https://spasibovsem.ru{review_link}', headers=header)
            review_page = BeautifulSoup(req_review.text, 'html.parser')
            rating = review_page.find('div', {'class': 'params'}).find('div', {'class': 'val'}).find('div', {'class': 'stars big'})['data-fill']
            review_text = review_page.find('div', {'class': 'text response-text description'}).get_text()
            all_info.loc[len(all_info.index)] = [item_name, review_text, rating]
            all_info.to_csv('data_movies.csv')

*По меры скачивания отзывов данные сохранялись в файл во избежании потери данных переменной при ошибках ядра Kernel*
<br>*На скачивание свыше 3к примеров ушло около 6 часов (такое кол-во примеров обусловлено предложениями по улучшению алгоритма (см. п. 5)*)

! Необходимые переменные я сохранила в файлы, т.к. их получение требует времени

In [2]:
import re
#собранные с сайта отзывы
data_full = pd.read_csv('data.csv', delimiter=',')
del data_full['Unnamed: 0']
data_full['Rate'] = data_full['Rate'].astype(int)
data_full['Real'] = data_full['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

#лемматизированные данные
data = pd.read_csv('all_data_tokens.csv', delimiter=',')
del data['Unnamed: 0']
data['Rate'] = data['Rate'].astype(int)
data['Real'] = data['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

#при сохранении файла меняется тип данных в колонке Review, поэтому из строки нужно опять сделать список
def clean_tokens(text):
    text2 = []
    for i in range(len(text)):
        text[i] = re.sub("'|«|»|,| ,|, ,| ", '', text[i])
        if text[i].isalpha():
          text2.append(text[i])
    return text2

data['Review'] = data['Review'].apply(lambda x: x[1:-2].split("',"))
data['Review'] = data['Review'].apply(clean_tokens)
data = data[data['Rate'] != 3]

In [15]:
data_full.head(10)

,Name,Review,Rate,Real
0,"Фильм ""Рассмеши меня"" (2023)",\r\nРоссийские комедии зачастую оказываются ве...,3,отрицательный
1,"Фильм ""Неудержимые 4"" (2023)","\r\nКрутые боевики всегда привлекают внимание,...",4,положительный
2,"Фильм ""За Палыча!"" (2023)","\r\nЛюблю глянуть комедий, а если на ролях зая...",5,положительный
3,"Фильм ""Тайны герцогини"" (2022)",\r\nИногда у людей могут складываться странные...,4,положительный
4,"Фильм ""У людей так бывает"" (2023)","\r\nЯ раньше никогда не смотрела кино, которое...",3,отрицательный
5,"Фильм ""Американские преступники"" (2023)","\r\nИногда случается так, что некоторым престу...",4,положительный
6,"Фильм ""Эйми"" (2023)",\r\nТема похищения и продажи людей всегда акту...,3,отрицательный
7,"Фильм ""Ставок больше нет"" (2023)","\r\nВсем давно известно, что увлечение азартны...",4,положительный
8,"Фильм ""Статистическая вероятность любви с перв...","\r\nДумаю многие верят в то, что есть судьбоно...",4,положительный
9,"Фильм ""Пенсионный план"" (2023)","\r\nКровные узы, безусловно, играют свою роль ...",3,отрицательный


*Review - текст отзыва, Rate - оценка по 5-ти балльной системе, Name - название фильма (эта колонка не нужна для анализа, ее было удобно создавать на этапе сбора данных, чтобы не запутаться).*

Можно удалить отзывы с оценкой 3, т.к. невозможно достоверно подготовить исходный размеченный датасет, поделив отзывы с Rate = 3 на положительные/отрицательные

In [3]:
data_full = data_full[data_full.Rate != 3]

In [18]:
data_full.groupby('Real').sum()

<ipython-input-18-197814b6922d>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_full.groupby('Real').sum()


,Rate
Real,
отрицательный,564
положительный,10562


Для устранения диспропорции выборки можно сделать следующее:

In [4]:
pos_df = data_full[data_full['Real'] == 'положительный'].sample(n=500)
neg_df = data_full[data_full['Real'] == 'отрицательный']
data_sample = pd.concat([pos_df, neg_df])

### **2. Препроцессинг**

На этом этапе проводим нормализацию данных, а именно:
- токенизируем предложения отзывов (с помощью библиотеки NLTK)
- приводим слова к нижнему регистру
- приводим слова к начальной форме (с помощью библиотеки pymorphy2)

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('russian'))
from nltk.tokenize import word_tokenize
from string import punctuation
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

def get_clean(text):
    clean_text = nltk.word_tokenize(text)
    lemmas = []
    for word in clean_text:
        if word not in punctuation and word.isalpha():
          word = word.lower()
          if morph.parse(word)[0].normal_form:
              lemmas.append(morph.parse(word)[0].normal_form)
    return lemmas

C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\602\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\602\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
data_sample['Review'] = data_sample['Review'].apply(get_clean)

In [10]:
data_sample

,Name,Review,Rate,Real
1509,"Фильм ""Король придурков"" (1981)","[забавный, старый, комедия, режиссёр, клод, ко...",4,положительный
1400,"Фильм ""Свингеры"" (2021)","[приветствовать, весь, заглянуть, в, этот, отз...",4,положительный
887,"Фильм ""Белый медведь"" (1998)","[фильм, белый, медведь, это, супер, ретро, бое...",5,положительный
1203,"Фильм ""Блестяще"" (2017)","[чудесный, комедийный, драма, иван, атталя, бл...",5,положительный
1949,"Фильм ""Фарго"" (1995)","[фарго, можно, смело, назвать, культовый, кино...",5,положительный
...,...,...,...,...
3336,"Фильм ""Доктор Сон"" (2019)","[фильм, доктор, сон, снятой, по, одноимённый, ...",1,отрицательный
3347,"Фильм ""Они"" (2019)","[добрый, время, сутки, сюжет, фильм, они, разв...",2,отрицательный
3363,"Фильм ""Терминатор: Тёмные судьбы"" (2019)","[в, последний, раз, я, писать, отзыв, на, новы...",2,отрицательный
3366,"Фильм ""Терминатор: Тёмные судьбы"" (2019)","[я, сожалеть, о, потратить, деньга, а, самый, ...",2,отрицательный


### **3. Тональный словарь**

Составляем 2 множества:
- слова, которые встречаются только в положительных отзывах
- слова, которые встречаются только в отрицательных отзывах

In [11]:
positive = data_sample[data_sample['Rate'] >= 4]['Review'].tolist()
negative = data_sample[data_sample['Rate'] < 4]['Review'].tolist()
positive = [element for each_list in positive for element in each_list]
negative = [element for each_list in negative for element in each_list]
only_positive = [x for x in positive if x not in negative]
only_negative = [x for x in negative if x not in positive]

Также удаляем слова, которые встречаются 1-2 раза, поскольку их попадание в множество может быть случайным.

In [12]:
only_positive_freq = []
for word in only_positive:
  if only_positive.count(word) > 2:
    only_positive_freq.append(word)

only_negative_freq = []
for word in only_negative:
  if only_negative.count(word) > 2:
    only_negative_freq.append(word)

only_positive_freq = set(only_positive_freq)
only_negative_freq = set(only_negative_freq)

In [13]:
[len(only_positive_freq), len(only_negative_freq)]

[666, 274]

В словаре наблюдается диспропорция, поэтому максимальное кол-во слов, характеризующих только отрицательные отзывы, не превышает 350 (при около 300 примерах, однако в таком случае выборка получается слишком маленькой, а словарь — непоказательным: мы пытаемся отсечь пересечение двух маленьких выборок, чем больше выборка, тем больше пересечений, тем больше вероятность того, что слово не попадет в словарь. Если у нас слишком много слов в словаре, велика вероятность того, что эти слова попали туда случайно и могут встретиться в полярном отзыве, просто на маленькой выборке алгоритм этого не мог заметить) --> при пропорциональной выборке оптимальным представляется кол-во примеров в 1к (про непропорциональную см. п. 5.2).

### **4. Анализ отзывов**

In [14]:
def sentiment(text):
  positive_count = int()
  negative_count = int()
  for lemma in text:
    if lemma in only_positive_freq:
      positive_count += 1
    elif lemma in only_negative_freq:
      negative_count += 1
  if positive_count > negative_count:
    return 'положительный'
  elif positive_count < negative_count:
    return 'отрицательный'
  else:
    return 'нейтральный'

Теперь посчитаем точность алгоритма для рандомных выборок одного размера:

Пример 1:

In [16]:
data_sample = data.sample(frac=0.2)
data_sample['Estimate'] = data_sample['Review'].apply(sentiment)
#добавим разметку на основе звезд, присужденных каждому отзыву пользователем
data_sample['Real'] = data_sample['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat = {'неверно': 0, 'верно': 0}
for index, row in data_sample.iterrows():
    if row['Estimate'] != row['Real']:
      stat['неверно'] += 1
    else:
      stat['верно'] += 1

print('Верно:', stat['верно'], 'Неверно:', stat['неверно'], 'Процент точности:', round(stat['верно']/(stat['верно']+stat['неверно'])*100, 4))

Верно: 454 Неверно: 76 Процент точности: 85.6604


Пример 2:

In [19]:
data_sample = data.sample(frac=0.2)
data_sample['Estimate'] = data_sample['Review'].apply(sentiment)
#добавим разметку на основе звезд, присужденных каждому отзыву пользователем
data_sample['Real'] = data_sample['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat = {'неверно': 0, 'верно': 0}
for index, row in data_sample.iterrows():
    if row['Estimate'] != row['Real']:
      stat['неверно'] += 1
    else:
      stat['верно'] += 1

print('Верно:', stat['верно'], 'Неверно:', stat['неверно'], 'Процент точности:', round(stat['верно']/(stat['верно']+stat['неверно'])*100, 4))

Верно: 440 Неверно: 90 Процент точности: 83.0189


--> точность по-разному проявляется на разных выборках, варьируется от 75 до 85% (также учитывались выборки из бОльшего числа примеров, точность примерно одинаковая)

А если выборка пропорциональна?

In [20]:
pos_df = data[data['Real'] == 'положительный'].sample(n=250)
neg_df = data[data['Real'] == 'отрицательный'].sample(n=250)
data_sample = pd.concat([pos_df, neg_df])
data_sample['Estimate'] = data_sample['Review'].apply(sentiment)
#добавим разметку на основе звезд, присужденных каждому отзыву пользователем
data_sample['Real'] = data_sample['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat = {'неверно': 0, 'верно': 0}
for index, row in data_sample.iterrows():
    if row['Estimate'] != row['Real']:
      stat['неверно'] += 1
    else:
      stat['верно'] += 1

print('Верно:', stat['верно'], 'Неверно:', stat['неверно'], 'Процент точности:', round(stat['верно']/(stat['верно']+stat['неверно'])*100, 4))

Верно: 415 Неверно: 85 Процент точности: 83.0


### **5. Улучшение алгоритма**

*Что я тестировала для повышения точности алгоритма:*

**1) Использование модели классификации**

Несмотря на то что мы избавились от возможно нейтральных отзывов с оценкой 3, алгоритм вcё же не может оценить некоторые отзывы и определяет их как "нейтральные" (хотя спорные отзывы, кажется, должны отсутствовать). Можно попробовать создать ***модель классификации***, которая бы сама разбиралась бы с "нейтральными" отзывами

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

data2 = data.copy()
data2['Review'] = data2['Review'].apply(lambda x: ' '.join(x))
d_train, d_test, y_train, y_test = train_test_split(data2['Review'], data2['Real'], train_size=0.8, random_state=0) #обучаем модель на около 2к примерах
vec = TfidfVectorizer(binary=True, max_features=1000)
vec.fit_transform(d_train)
X_train = vec.transform(d_train).astype(int)
X_test = vec.transform(d_test).astype(int)
Y_train = vec.transform(y_train).astype(int)
Y_test = vec.transform(y_test).astype(int)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8924528301886793

Пример 1:

In [23]:
data_test = data.sample(frac=0.2)
data_test['Estimate'] = data_test['Review'].apply(sentiment)
data_test['Review'] = data_test['Review'].apply(lambda x: ' '.join(x))

for index, row in data_test.iterrows():
    if row['Estimate'] == 'нейтральный':
        data_test.at[index, 'Estimate'] = model.predict(vec.transform([row['Review']]).astype(int))[0]

y_pred = vec.transform(data_test['Estimate']).astype(int)
y_test = vec.transform(data_test['Real']).astype(int)
accuracy_score(y_test, y_pred)

0.9433962264150944

Пример 2 (Пропорциональная выборка):

In [28]:
pos_df = data[data['Real'] == 'положительный'].sample(n=250)
neg_df = data[data['Real'] == 'отрицательный'].sample(n=250)
data_test = pd.concat([pos_df, neg_df])
data_test['Estimate'] = data_test['Review'].apply(sentiment)
data_test['Review'] = data_test['Review'].apply(lambda x: ' '.join(x))

for index, row in data_test.iterrows():
    if row['Estimate'] == 'нейтральный':
        data_test.at[index, 'Estimate'] = model.predict(vec.transform([row['Review']]).astype(int))[0]

y_pred = vec.transform(data_test['Estimate']).astype(int)
y_test = vec.transform(data_test['Real']).astype(int)
accuracy_score(y_test, y_pred)

0.912

Также можно пробовать уже готовые модели из библиотек. Например, *TextBlob*:

In [29]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
nltk.download('brown')

def neutral(text):
    positive_count = int()
    negative_count = int()
    text_blob_object = TextBlob(text)
    for noun_phrase in text_blob_object.noun_phrases:
        if (TextBlob(noun_phrase).sentiment[0]) < 0:
          negative_count += 1
        elif (TextBlob(noun_phrase).sentiment[0]) > 0.5:
          positive_count += 1
    if positive_count > negative_count:
      return 'положительный'
    elif positive_count < negative_count:
      return 'отрицательный'
    else:
      return 'нейтральный'

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\602\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


В этом примере TextBlob выделяет именные группы, после чего определяется тональность каждой из них.

In [30]:
data3 = data.sample(frac=0.2)
data3['Estimate'] = data3['Review'].apply(sentiment)
data3['Review'] = data3['Review'].apply(lambda x: ' '.join(x))
for index, row in data3.iterrows():
    if row['Estimate'] == 'нейтральный':
        data3.at[index, 'Estimate'] = neutral(row['Review'])

stat3 = {'неверно': 0, 'верно': 0}
for index, row in data3.iterrows():
    if row['Estimate'] != row['Real']:
      stat3['неверно'] += 1
    else:
      stat3['верно'] += 1

print('Верно:', stat3['верно'], 'Неверно:', stat3['неверно'], 'Процент точности:', round(stat3['верно']/(stat3['верно']+stat3['неверно'])*100, 2))

Верно: 449 Неверно: 81 Процент точности: 84.72


Однако этот вариант практически не меняет точность

**2) Расширение тонального словаря путем расширения выборки**

Полная выборка состоит из около 3.3к примеров, поэтому попробуемм обработать весь датасет:

In [ ]:
data4 = data.copy()
positive = data4[data4['Rate'] >= 4]['Review'].tolist()
negative = data4[data4['Rate'] < 4]['Review'].tolist()
positive = [element for each_list in positive for element in each_list]
negative = [element for each_list in negative for element in each_list]
only_positive = [x for x in positive if x not in negative]
only_negative = [x for x in negative if x not in positive]

In [83]:
only_positive_freq = []
for word in only_positive:
  if only_positive.count(word) > 2:
    only_positive_freq.append(word)

only_negative_freq = []
for word in only_negative:
  if only_negative.count(word) > 2:
    only_negative_freq.append(word)

only_positive_freq = set(only_positive_freq)
only_negative_freq = set(only_negative_freq)

In [84]:
[len(only_positive_freq), len(only_negative_freq)]

[3975, 62]

Опять же наблюдается диспропорция, причем чем больше кол-во примеров, тем больше перекос в сторону положительных отзывов (поскольку увеличивается непосредственно кол-во положительных отзывов). В дальнейшем эту ситуацию можно будет исправить добавлением бОльшего количества данных с отрицательной окраской (на сайте, с которого были спарсены все имеющиеся там отзывы на фильмы, к сожалению, наблюдается перевес в сторону положительных отзывов).

In [85]:
data4 = data.sample(frac=0.3)
data4['Estimate'] = data4['Review'].apply(sentiment)
data4['Real'] = data4['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat4 = {'неверно': 0, 'верно': 0}
for index, row in data4.iterrows():
    if row['Estimate'] != row['Real']:
      stat4['неверно'] += 1
    else:
      stat4['верно'] += 1

print('Верно:', stat4['верно'], 'Неверно:', stat4['неверно'], 'Процент точности:', round(stat4['верно']/(stat4['верно']+stat4['неверно'])*100, 4))


Верно: 726 Неверно: 68 Процент точности: 91.4358


Пропорциональная выборка:

In [91]:
pos_df = data[data['Real'] == 'положительный'].sample(n=250)
neg_df = data[data['Real'] == 'отрицательный'].sample(n=250)
data4 = pd.concat([pos_df, neg_df])
data4['Estimate'] = data4['Review'].apply(sentiment)
data4['Real'] = data4['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat4 = {'неверно': 0, 'верно': 0}
for index, row in data4.iterrows():
    if row['Estimate'] != row['Real']:
      stat4['неверно'] += 1
    else:
      stat4['верно'] += 1

print('Верно:', stat4['верно'], 'Неверно:', stat4['неверно'], 'Процент точности:', round(stat4['верно']/(stat4['верно']+stat4['неверно'])*100, 4))


Верно: 329 Неверно: 171 Процент точности: 65.8


Таким образом, результат точности на пропорциональной выборке достаточно плох. Значит, необходимо что-то сделать со словарем, добавив туда слова с отрицательной окраской. Если таких слов не хватает в собственной выборке, необходимо либо парсить другие сайты либо воспользоваться готовым словарем (см. следующий способ).

**3) Расширение тонального словаря с помощью готовых размеченных словарей**

Например, можно использовать [Карту слов](https://github.com/dkulagin/kartaslov?ysclid=lmtihzuzyw822019313).


In [32]:
sent_dict = pd.read_csv('kartaslovsent.csv', delimiter=';')

In [33]:
sent_dict

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


In [34]:
def sentiment2(text):
  positive_count = int()
  negative_count = int()
  for lemma in text:
    if lemma in only_positive_freq:
      positive_count += 1
    elif lemma in only_negative_freq:
      negative_count += 1
    elif lemma in sent_dict['term']:
      if sent_dict[sent_dict['term'] == lemma]['tag'] == 'PSTV':
        positive_count += 1
      elif sent_dict[sent_dict['term'] == lemma]['tag'] == 'NGTV':
        negative_count += 1

  if positive_count > negative_count:
    return 'положительный'
  elif positive_count < negative_count:
    return 'отрицательный'
  else:
    return 'нейтральный'

Возвращаемся к тональному словарю прежнего размера:

In [ ]:
pos_df = data_full[data_full['Real'] == 'положительный'].sample(n=500)
neg_df = data_full[data_full['Real'] == 'отрицательный']
data_sample = pd.concat([pos_df, neg_df])
data_sample['Review'] = data_sample['Review'].apply(get_clean)

positive = data_sample[data_sample['Rate'] >= 4]['Review'].tolist()
negative = data_sample[data_sample['Rate'] < 4]['Review'].tolist()
positive = [element for each_list in positive for element in each_list]
negative = [element for each_list in negative for element in each_list]
only_positive = [x for x in positive if x not in negative]
only_negative = [x for x in negative if x not in positive]

only_positive_freq = []
for word in only_positive:
  if only_positive.count(word) > 2:
    only_positive_freq.append(word)

only_negative_freq = []
for word in only_negative:
  if only_negative.count(word) > 2:
    only_negative_freq.append(word)

only_positive_freq = set(only_positive_freq)
only_negative_freq = set(only_negative_freq)

In [135]:
[len(only_positive_freq), len(only_negative_freq)]

[616, 270]

In [134]:
pos_df = data[data['Real'] == 'положительный'].sample(n=250)
neg_df = data[data['Real'] == 'отрицательный'].sample(n=250)
data5 = pd.concat([pos_df, neg_df])
# data5 = data.sample(frac=0.2)
data5['Estimate'] = data5['Review'].apply(sentiment)
data5['Real'] = data5['Rate'].apply(lambda x: 'положительный' if x > 3 else 'отрицательный')

stat5 = {'неверно': 0, 'верно': 0}
for index, row in data5.iterrows():
    if row['Estimate'] != row['Real']:
      stat5['неверно'] += 1
    else:
      stat5['верно'] += 1

print('Верно:', stat5['верно'], 'Неверно:', stat5['неверно'], 'Процент точности:', round(stat5['верно']/(stat5['верно']+stat5['неверно'])*100, 3))

Верно: 431 Неверно: 69 Процент точности: 86.2


Заметим, что, несмотря на то что точность не сильно повысилась, алгоритм стал устойчивее (точность теперь не опускается ниже 82%).

**4) Удаление стоп-слов из списка лемм**

In [121]:
only_positive_freq2 = [x for x in only_positive_freq if x not in list(stops)]
only_negative_freq2 = [x for x in only_negative_freq if x not in list(stops)]

In [122]:
[len(only_positive_freq2), len(only_negative_freq2)]

[616, 270]

Однако этот метод ни к чему не привел, поскольку, вероятнее всего, стоп-слова находятся на пересечении двух категорий отзывов (встречаются как в положительных, так и в отрицательных отзывах), поэтому объем словаря не меняется --> не меняется точность.

### **6. Разные модели классификации**

**На этапе векторизации были использованы следующие векторизаторы:**
<p> -- CountVectorizer
<p> -- TF-IDF

**Модели, которые использовались для задачи классификации:**
<p> -- Логистическая регрессия
<p> -- DecisionTreeClassifier (дерево решений)
<p> -- RandomForestClassifier (алгоритм случайного дерева)
<p> -- KNeighborsClassifier (метод ближайших соседей)
<p> -- MLPClassifier (многослойный перцептрон)
<p> -- SVC (метод опорных векторов)


6.1. TF-IDF

In [71]:
pos_df = data[data['Real'] == 'положительный'].sample(n=400)
neg_df = data[data['Real'] == 'отрицательный']
data5 = pd.concat([pos_df, neg_df])
data5['Review'] = data5['Review'].apply(lambda x: ' '.join(x))
d_train, d_test, y_train, y_test = train_test_split(data5['Review'], data5['Real'], train_size=0.8, random_state=0) #обучаем модель на около 2к примерах
vec = TfidfVectorizer(binary=True, max_features=1000)
vec.fit_transform(d_train)
X_train = vec.transform(d_train).astype(int)
X_test = vec.transform(d_test).astype(int)
Y_train = vec.transform(y_train).astype(int)
Y_test = vec.transform(y_test).astype(int)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [72]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=35)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [73]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(criterion='gini', min_samples_leaf=1, n_estimators=200, max_depth=100, random_state=2).fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [74]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=200)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [75]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-05, hidden_layer_sizes=(350, 2), random_state=1).fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [76]:
from sklearn.svm import SVC
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)
y_pred = svc_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

2. CountVectorizer + LogisticRegression

In [45]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score

cvec = CountVectorizer(binary=True, max_features=10000)
cvec.fit(d_train)
X_train = cvec.transform(d_train).astype(int)
X_test = cvec.transform(d_test).astype(int)
feature_names = cvec.get_feature_names()

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0.85

In [54]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=35)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
accuracy_score(y_test, y_pred)

0.7

In [55]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(criterion='gini', min_samples_leaf=1, n_estimators=200, max_depth=100, random_state=2).fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy_score(y_test, y_pred)

0.83

In [56]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=200)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

0.73

In [80]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-05, hidden_layer_sizes=(200, 2), random_state=1).fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.54421768707483

In [58]:
from sklearn.svm import SVC
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)
y_pred = svc_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.82

Таким образом, наиболее точные результаты выдает модель CountVectorizer + LogisticRegression (85%). Векторизация с помощью TF-IDF в сочетании с любыми из рассматриваемых моделей дает плохие результаты.